In [2]:
import pandas as pd
import json

In [3]:
train_img_path = '../../dataset/train/*'
train_json_path = '../../dataset/train.json'

test_img_path = '../../dataset/test/*'
test_json_path = '../../dataset/test.json'

model_labeled_csv_path = '../mmdetection/work_dirs/faster_rcnn_res50/submission_latest.csv'

In [15]:
df = pd.read_csv(model_labeled_csv_path)

df

,PredictionString,image_id
0,0 0.17261261 237.58946 672.7023 299.2572 748.2...,test/0000.jpg
1,0 0.4511335 757.7669 659.82947 895.48126 986.0...,test/0001.jpg
2,0 0.25431386 877.1926 472.16486 1017.22363 635...,test/0002.jpg
3,0 0.28635767 54.320282 134.41394 1005.4558 868...,test/0003.jpg
4,0 0.5941855 186.45221 346.1676 897.1718 778.32...,test/0004.jpg
...,...,...
4866,0 0.4447822 294.58755 344.97754 669.4094 690.5...,test/4866.jpg
4867,3 0.95047486 411.95758 316.81677 779.85333 700...,test/4867.jpg
4868,0 0.22523004 545.2088 527.6001 715.26056 824.6...,test/4868.jpg
4869,0 0.6614299 13.801971 181.19055 1002.3755 938....,test/4869.jpg


In [ ]:
# --train json 파일 확인

with open(train_json_path, 'r') as f:
    json_data = json.load(f)
print(json.dumps(json_data, indent='\t'))

In [32]:
# --model이 예측한 csv 파일 확인 및 리스트에 정보 담기

df = pd.read_csv(model_labeled_csv_path)
df = df.dropna()
print(df.isnull().sum())

anns_list = list()
id = 0
for x in df.index:
    image_name = df.loc[x][1].split('/')[1]
    image_id = int(image_name.split('.')[0])
    information = list(df.loc[x][0].split())
    for i in range(0, len(information), 6):
        label = int(information[i])
        score = float(information[i+1])
        xmin = round(float(information[i+2]), 1)
        ymin = round(float(information[i+3]), 1)
        xmax = round(float(information[i+4]), 1)
        ymax = round(float(information[i+5]), 1)
        w = round(xmax - xmin, 2)
        h = round(ymax - ymin, 2)
        area = round(w*h, 2)

        if score > 0.3:
            ann = dict()
            ann['image_id'] = image_id
            ann['category_id'] = label
            ann['area'] = area
            ann['bbox'] = [xmin, ymin, w, h]
            ann['iscrowd'] = 0
            ann['id'] = id
            id += 1
            anns_list.append(ann)

PredictionString    0
image_id            0
dtype: int64


In [33]:
# --pseudo label json 파일 만들기

save_file_path = '../../dataset/pseudo.json'
with open(test_json_path, 'r') as f:
    json_data = json.load(f)

json_data['annotations'] = anns_list

with open(save_file_path, 'w', encoding='utf-8') as file:
    json.dump(json_data, file, indent='\t')